In [1]:
"""
General workflow for importing a new session
"""
subject = 'P07' # TODO: change this for each subject
verbose = True  
# change this for debugging 
# If True, the time elapsed while fitting each step will be printed as it is completed.  

import matplotlib
%matplotlib inline


# from deepthought.datasets.openmiir.preprocessing.pipeline import Pipeline # Previous job
from sklearn.pipeline import Pipeline

settings = dict(debug=False,
                mne_log_level='Info',
                sfreq=64) # optional pipeline settings

# pipeline = Pipeline(subject, settings)  # optional pipeline settings

pipeline = Pipeline(['subject', 'settings']) # sklearn



ValueError: too many values to unpack

# Import or Load Raw

In [ ]:
pipeline.import_and_process_metadata(verbose=verbose)
# 这句话本来没有被注释掉，是高茂森注释掉的

In [ ]:

# IOError: [Errno 2] No such file or directory: 'E:\\OpenMIIR-RawEEG_v1\\OpenMIIR\\eeg\\biosemi\\P07.bdf'
pipeline.load_raw(verbose=verbose)
# [Errno 2] No such file or directory: 'E:\\OpenMIIR-RawEEG_v1\\OpenMIIR\\eeg\\mne\\P07-raw.fif'

In [ ]:
print pipeline.raw.info

# Reject Bad Channels

In [ ]:
# switch to interactive GUI mode to scroll through data
%matplotlib tk 
pipeline.plot_raw();
pass

In [ ]:
# switch back to inline mode
%matplotlib inline

In [ ]:
# pipeline.reset_bad_channels() # use this to reset the channel list if needed

In [ ]:
# TODO: change these value manually
# If you would rather apply the bandpass filter first, continue until it is applied 
# and then come back to this cell
pipeline.mark_bad_channels(None, save_to_raw=False) # nothing to change, None will keep old values
# pipeline.mark_bad_channels(['P8', 'P10', 'T8'], save_to_raw=True)

In [ ]:
pipeline.print_bad_channels()

In [ ]:
pipeline.interpolate_bad_channels() # Note: this will overwrite data

# Check and Merge Trials

In [ ]:
pipeline.check_trial_events()

In [ ]:
pipeline.check_trial_audio_onset_merge(use_audio_onsets=True, 
                                       verbose=None)

In [ ]:
pipeline.merge_trial_and_audio_onsets()

# Check PSD and Channels

In [ ]:
pipeline.check_psd()
pipeline.check_psd(fmax=35)
# line noise will probably be visible @ multiples of 60Hz

In [ ]:
pipeline.check_channel(0)
# quite some drift and movement in the breaks

# Bandpass Filtering

In [ ]:
pipeline.bandpass_filter()

In [ ]:
%matplotlib tk 
pipeline.plot_raw();
pass

In [ ]:
%matplotlib inline

In [ ]:
pipeline.check_channel(0)
# looks like we got rid of that drift - nice!
# what's that? eyeblinks?

In [ ]:
## check PSD again - after bandpass, before down-sampling
pipeline.check_psd()
pipeline.check_psd(fmax=35)
# 60Hz is still visible

# Beat and EOG Epoching

In [ ]:
pipeline.generate_beat_events() # Note: this includes cue-beats !!!
pipeline.beat_epochs.average().plot();

In [ ]:
pipeline.find_eog_events()
# wow - that's a lot of blinking

# Down-Sampling

In [ ]:
pipeline.downsample()

In [ ]:
## PSD after down-sampling:
pipeline.check_psd(fmax=35)
# looks less smooth than without down-sampling

In [ ]:
# check events after after resampling -> should get the same result as above
pipeline.check_resampled_trial_events(plot=True, verbose=False)

# ICA

In [ ]:
pipeline.compute_ica(verbose=True)

In [ ]:
%matplotlib inline
pipeline.plot_ica_components() # static plot

In [ ]:
pipeline.inspect_source_psd(1) # plot the PSD of an IC source to look for alpha activity
pipeline.inspect_source_psd(2)

In [ ]:
pipeline.find_eog_artifact_sources(plot=True, verbose=False)

In [ ]:
pipeline.auto_detect_artifact_components()

In [ ]:
pipeline.select_artifact_sources(selection='auto')

In [ ]:
pipeline.exclude_ica_components(selection=[0, 2]) # TODO: adapt selection list
# use cells below to decide, which components to reject, then come back to this cell

In [ ]:
for ic in pipeline.suggested_artifact_components:
# for ic in pipeline.ica.exclude:
    pipeline.inspect_ica_component(ic, [0, 700])
    pipeline.inspect_source_epochs(ic, mode='beats', vmax=5)
    pipeline.inspect_source_epochs(ic, mode='eog', vmax=5)

In [ ]:
# enable this for interactive mode
# %matplotlib tk

# pipeline.plot_sources(mode='raw')   # this may take a while
# pipeline.plot_sources(mode='beats')
pipeline.plot_sources(mode='eog');

In [ ]:
# switch back to non-interactive inline mode
%matplotlib inline

In [ ]:
pipeline.assess_unmixing_quality(verbose=False)

In [ ]:
pipeline.save_ica('100p_64c') # save for later

# Application Test

In [ ]:
pipeline.load_ica('100p_64c')

In [ ]:
raw = pipeline.ica.apply(pipeline.raw, exclude=(pipeline.ica.exclude), copy=False)
if len(raw.info['bads']) > 0:
    raw.interpolate_bads_eeg() # interpolate bad channels afterwards as they are not processed by the ICA

%matplotlib tk 
pipeline.plot_raw();
pass